In [ ]:
#model 2-1: homogeneous quality + homogeneous rhetoric
import random
import numpy as np
random.seed(100000)
np.random.seed(100000)
import scipy.stats as stats
from scipy.stats import pearsonr
from scipy.stats import norm

#helper function for calculating Gini coefficient
def gini_coefficient(x):
    """Compute Gini coefficient of array of values"""
    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        diffsum += np.sum(np.abs(xi - x[i:]))
    return diffsum / (len(x)**2 * np.mean(x))

#helper function for getting indices of the top N values of a list (which to read/cite)
def f(a,N):
    """Get indices of the top N values of a list"""
    return np.argsort(a)[::-1][:N]

#args you need to use
tmax= 1000#time steps
#-----------------------------
num = 600#paper population
nummax=600
#-----------------------------
reference = 40#expected reference size, need to vary
refmax =40
#-----------------------------
reading = 120#reading size, need to vary
readingmax=120
#-----------------------------
noise =0.05
#-----------------------------
shape=6

listcorr=[]
listgini=[]
threshold=0.5 #homogeneous threshold

for num in range(num, nummax+1):
  normative=np.random.beta(1, shape, size=num)
  qrank =[]
  qrank = list(np.argsort(normative)[-(num):][::-1])

#-------------------------------------------------------------------------------
#here is the major difference from model 1.1
#underlying rhetorical value is homogeneous and perceived equally
#thus it is outside the t loop (reader loop)
  base_rhe=np.random.beta(1, shape, size=num)
  rrank =[]
  rrank = list(np.argsort(base_rhe)[-(num):][::-1])

  #move to the model
  weight=0.001 #the weight of citation count on perceived quality
  weightq =0.3 #the signal-based gain in rhetoric value

  list1=[]
  list2=[]
  list3=[]
  list4=[]

  for reference in range(reference, refmax+1):
  #for reading in range(reading, readingmax+1):
    top1q=qrank[0:40]
    top2q=qrank[40:int(150)]

    cite_population = [0]*num #citation count over the entire paper population
    rhe_list=[0]*num #rhetoric value
    chunk =[]

    for t in range(1, tmax+1):

      noise_list=np.random.normal(0, noise, num)
      #here, noise is random, so it's within the loop of t (readers)

      signal =[]
      for i in range(num):
        signal.append(normative[i]+ weight * cite_population[i] + noise_list[i])
      #no fit -- because fit is specific for heterogeneous models

      for i in range(num):
        if(signal[i] >2):
          signal[i] =2
        if(signal[i] <0):
          signal[i]=0

      reading_index = list(np.argsort(signal)[-(reading):][::-1])

      for i in range(len(reading_index)):
        rhe_list[reading_index[i]] =base_rhe[reading_index[i]] + weightq*normative[reading_index[i]] + weightq* weight*cite_population[reading_index[i]]

      unread=[]
      unread = [i for i in list(range(0,num)) if i not in reading_index]
      for i in range(len(unread)):
        rhe_list[unread[i]] =base_rhe[unread[i]] + weightq*normative[unread[i]] + weightq* weight*cite_population[unread[i]] + weightq* noise_list[unread[i]]

      for i in range(num):
        if (rhe_list[i] <0):
          rhe_list[i] =0
        if(rhe_list[i] > 1+ weightq+ weightq):
          rhe_list[i] =1+ weightq+ weightq

      norm_list =[]
      for i in range(len(reading_index)):
        norm_list.append(normative[reading_index[i]])
      for i in range(len(reading_index)):
        if (norm_list[i] <0):
          norm_list[i] =0
        if (norm_list[i] >1):
          norm_list[i] =1

      over_threshold=[]
      for i in range(len(norm_list)):
        if (norm_list[i]> threshold):
          over_threshold.append(reading_index[i])

      normative_cite=[]# substantive citation list
      rhetorical_cite=[]# rhetorical citation list
      overlap=[]#overlap
      overall_cite =[]

      if (len(over_threshold) >= reference):
        cite = list(f(norm_list, reference))
        for i in range(len(cite)):
          cite_population[reading_index[cite[i]]]= cite_population[reading_index[cite[i]]]+1
          normative_cite.append(reading_index[cite[i]])
        rhetorical_cite=[]
        overlap =[]
        overall_cite = normative_cite + rhetorical_cite + overlap

      else:
        normative_cite = over_threshold.copy()
        rhetoric_no = reference - len(over_threshold)
        new_rhe = rhe_list.copy()
        rhe2=[]
        rhe2 = sorted(new_rhe, reverse = True)
        itr=0
        itr2=0
        while (itr < rhetoric_no):
          if ((new_rhe.index(rhe2[itr2]) in normative_cite) == True):
            normative_cite.remove(new_rhe.index(rhe2[itr2]))
            numitr = new_rhe.index(rhe2[itr2])
            overlap.append(numitr)
            itr2 =itr2+1
          else:
            rhetorical_cite.append(new_rhe.index(rhe2[itr2]))
            itr =itr+1
            itr2 =itr2+1
        overall_cite = normative_cite + rhetorical_cite + overlap
        for i in range(len(overall_cite)):
          cite_population[overall_cite[i]] = cite_population[overall_cite[i]] +1
      #churn
      chunk.append(overall_cite)

#here, experiments--------------------------------------------------------------
    #listgini.append(gini_coefficient(np.array(cite_population)))
    #corr1, _ = stats.pearsonr(normative, cite_population)
    #listcorr.append(corr1)

      #top1=0
      #for i in range(40):
      # if ((top1q[i] in (normative_cite + overlap)) == True):
      #    top1 = top1+1
      #list1.append(top1/reference)

      #top2=0
      #for i in range(40):
      #  if ((top1q[i] in (rhetorical_cite + overlap)) == True):
      #    top2 = top2+1
      #list2.append(top2/reference)

      #top3=0
      #for i in range(110):
      #  if ((top2q[i] in (normative_cite + overlap)) == True):
      #    top3 = top3+1
      #list3.append(top3/reference)

     # top4=0
      #for i in range(110):
      #   if ((top2q[i] in (rhetorical_cite + overlap)) == True):
      #    top4 = top4+1
      #list4.append(top4/reference)

    #dnew=[]
    #for i in range(len(chunk) -1):
    #  a=0
    #  for j in range(len(chunk[i+1])):
    #    if ((chunk[i+1][j] in chunk[i]) == False):
    #      a=a+1
    #  dnew.append(a)
    #print(np.mean(dnew))

#print(list1)
#print(list2)
#print(list3)
#print(list4)
#print(listcorr)
#print(listgini)
print('example output: the citation distribution after 1000 time steps:')
print(*cite_population, sep='\n')

example output: the citation distribution after 1000 time steps:
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
0
0
0
0
1000
0
0
0
64
0
0
0
1000
0
0
0
0
0
0
0
0
0
1000
0
0
0
0
0
0
0
0
996
1000
0
0
0
0
0
0
0
0
0
0
995
0
0
0
0
0
0
0
0
1000
0
0
0
0
0
0
0
0
0
0
0
1000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
0
0
0
0
0
1000
0
0
0
0
0
0
0
0
0
0
0
1000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
972
0
0
0
0
0
0
0
0
1000
0
0
0
1000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
1000
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
980
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
0
0
0
1000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
994
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
0
0
0
0
1000
0
0
0
0
0
0
0
0
0
1000
0
0
0
0
1000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000
0
0
0
0
0
0
0
0
0
0
0
0
1000
0
0
0
0
0
999
0
0
0
0

In the homogeneous model, after 1000 iterations, the distribution is nearly bimodal with some papers having 0 citations and some 1000. Given that citation distributions in practice are never bimodal, we do not believe such a simple model is sufficiently realistic. For completeness, we present all three community health metrics for the three versions of this model (Full, Null-reference, Null-threshold) in Appendix Figure S13, but do not interpret it further.